# RNN Exemple pour l'onde sinusoïdale

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Données

Utilisons Numpy pour créer une simple onde sinusoïdale.

In [ ]:
x = np.linspace(0,50,501)
y = np.sin(x)

In [ ]:
x

In [ ]:
y

Transformons cela en un DataFrame

In [ ]:
df = pd.DataFrame(data=y,index=x,columns=['Sinus'])

In [ ]:
df.head(50)

## Répartition Train / Test

Note ! C'est très différent de notre méthodologie habituelle de répartition Entraînement / Test !

In [ ]:
len(df)

In [ ]:
test_percent = 0.1

In [ ]:
len(df)*test_percent

In [ ]:
test_point = np.round(len(df)*test_percent)

In [ ]:
test_ind = int(len(df) - test_point)

In [ ]:
test_ind

In [ ]:
train = df.iloc[:test_ind]
test = df.iloc[test_ind:]

In [ ]:
train

In [ ]:
test

## Mise à l'échelle des données

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(train)

In [ ]:
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

## Générateur de séries temporelles

Cette classe examine une série de points de données recueillis à
des intervalles égaux, ainsi que des paramètres de séries temporelles tels stride, length , etc., afin de produire des lots pour
l'entraînement/validation.

#### Arguments
    data: Indexable generator (such as list or Numpy array)
        containing consecutive data points (timesteps).
        The data should be at 2D, and axis 0 is expected
        to be the time dimension.
    targets: Targets corresponding to timesteps in `data`.
        It should have same length as `data`.
    length: Length of the output sequences (in number of timesteps).
    sampling_rate: Period between successive individual timesteps
        within sequences. For rate `r`, timesteps
        `data[i]`, `data[i-r]`, ... `data[i - length]`
        are used for create a sample sequence.
    stride: Period between successive output sequences.
        For stride `s`, consecutive output samples would
        be centered around `data[i]`, `data[i+s]`, `data[i+2*s]`, etc.
    start_index: Data points earlier than `start_index` will not be used
        in the output sequences. This is useful to reserve part of the
        data for test or validation.
    end_index: Data points later than `end_index` will not be used
        in the output sequences. This is useful to reserve part of the
        data for test or validation.
    shuffle: Whether to shuffle output samples,
        or instead draw them in chronological order.
    reverse: Boolean: if `true`, timesteps in each output sample will be
        in reverse chronological order.
    batch_size: Number of timeseries samples in each batch
        (except maybe the last one).

In [ ]:
scaled_train

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# définir un générateur
length = 2 # Longueur des séquences de sortie (en nombre de pas temporel)
batch_size = 1 # Nombre d'échantillons de séries temporelles dans chaque batch (lot)
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)

In [ ]:
len(scaled_train)

In [ ]:
len(generator) # n_input = 2

In [ ]:
# À quoi ressemble le premier batch ?
X,y = generator[0]

In [ ]:
print(f"Compte tenu du tableau suivant : \n{X.flatten()}")
print(f'Cela prédit ce y : \n {y}')

In [ ]:
# Redéfinissons pour obtenir 10 pas en arrière et prédisons ensuite le prochain pas
length = 10 # Longueur des séquences de sortie (en nombre de pas temporel)
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=1)

In [ ]:
# À quoi ressemble le premier batch ?
X,y = generator[0]

In [ ]:
print(f"Compte tenu du tableau suivant: \n{X.flatten()}")
print(f'Cela prédit ce y : \n {y}')

In [ ]:
length = 50 # Longueur des séquences de sortie (en nombre de pas temporel)
generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=1)

In [ ]:
X,y = generator[0]
print(f"Compte tenu du tableau suivant: \n{X.flatten()}")
print(f'Cela prédit ce y : \n {y}')

Vous allez maintenant pouvoir modifier la longueur de manière à ce qu'elle ait un sens pour votre série temporelle !

## Créer le modèle

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN

In [ ]:
# Nous n'utilisons qu'une seule feature dans notre série temporelle
n_features = 1

In [ ]:
# définir le modèle
model = Sequential()

# Couche simple RNN
model.add(SimpleRNN(50,input_shape=(length, n_features)))

# Prédiction finale
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
# ajustement du modèle
model.fit(generator,epochs=5)

In [ ]:
model.history.history.keys()

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

## Évaluer sur les données de test

In [ ]:
first_eval_batch = scaled_train[-length:]

In [ ]:
first_eval_batch

In [ ]:
first_eval_batch = first_eval_batch.reshape((1, length, n_features))

In [ ]:
model.predict(first_eval_batch)

In [ ]:
scaled_test[0]

Mettons maintenant cette logique dans une boucle for pour prédire l'avenir pour toute la gamme de test.

----

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

In [ ]:
current_batch.shape

In [ ]:
current_batch

In [ ]:
np.append(current_batch[:,1:,:],[[[99]]],axis=1)

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # obtenir la prédiction avec 1 timestamp d'avance ([0] pour ne saisir que le nombre au lieu de [array])
    current_pred = model.predict(current_batch)[0]
    
    # stocker la prédiction
    test_predictions.append(current_pred) 
    
    # mise à jour du batch pour inclure maintenant la prédiction et supprimer la première valeur
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
scaled_test

## Transformations inverses et Comparaison

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
true_predictions

In [ ]:
test

In [ ]:
# Ignorez le warning
test['Predictions'] = true_predictions

In [ ]:
test

In [ ]:
test.plot(figsize=(12,8))

## Ajout d'un Early Stopping et d'un Générateur de Validation

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
length = 49
generator = TimeseriesGenerator(scaled_train,scaled_train,
                               length=length,batch_size=1)


validation_generator = TimeseriesGenerator(scaled_test,scaled_test,
                                          length=length,batch_size=1)

## LSTMS

In [ ]:
# définir le modèle
model = Sequential()

# Couche LSTM
model.add(LSTM(50,input_shape=(length, n_features)))

# Prédiction finale
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(generator,epochs=20,
          validation_data=validation_generator,
          callbacks=[early_stop])

In [ ]:
model.summary()

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(len(test)):
    
    # obtenir la prédiction avec 1 timestamp d'avance ([0] pour ne saisir que le nombre au lieu de [array])
    current_pred = model.predict(current_batch)[0]
    
    # stocker la prédiction
    test_predictions.append(current_pred) 
    
    # mise à jour du batch pour inclure maintenant la prédiction et supprimer la première valeur
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
# Ignorez le warning
true_predictions = scaler.inverse_transform(test_predictions)
test['LSTM Predictions'] = true_predictions
test.plot(figsize=(12,8))

## Prévisions

Prévisions dans un intervalle inconnu. Nous devrions d'abord utiliser toutes nos données, puisque nous faisons maintenant des prévisions !

In [ ]:
full_scaler = MinMaxScaler()
scaled_full_data = full_scaler.fit_transform(df)

In [ ]:
length = 50 # Longueur des séquences de sortie (en nombre de pas temporel)
generator = TimeseriesGenerator(scaled_full_data, scaled_full_data, length=length, batch_size=1)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(length, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(generator,epochs=6)

In [ ]:
forecast = []

first_eval_batch = scaled_full_data[-length:]
current_batch = first_eval_batch.reshape((1, length, n_features))

for i in range(100):
    
    # obtenir la prédiction avec 1 timestamp d'avance ([0] pour ne saisir que le nombre au lieu de [array])
    current_pred = model.predict(current_batch)[0]
    
    # stocker la prédiction
    forecast.append(current_pred) 
    
    # mise à jour du batch pour inclure maintenant la prédiction et supprimer la première valeur
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
forecast = full_scaler.inverse_transform(forecast)

In [ ]:
forecast

In [ ]:
df

In [ ]:
len(forecast)

In [ ]:
50*0.1

In [ ]:
forecast_index = np.arange(50.1,60.1,step=0.1)

In [ ]:
len(forecast_index)

In [ ]:
plt.plot(df.index,df['Sinus'])
plt.plot(forecast_index,forecast)